In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet_FC'
# NETWORK = 'Resnet'
# NETWORK = 'Resnet_res2c'
# NETWORK = 'Resnet_res3d'
# NETWORK = 'Resnet_res4f'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'EfficientNetV2M'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ResNet152V2'
NETWORK = 'ConvNeXtBase'

METHOD = "Max-Median"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'

PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"

features_path = os.path.join(WORKING_DIRECTORY, "ExtractedFeatures")
processed_features_path = os.path.join(WORKING_DIRECTORY, "ProcessedFeatures")

processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,1.587701,1.876628,2.530490,2.637568,2.058125,1.453559,1.032105,1.939603,1.060274,...,1.818623,1.717797,1.650555,1.965334,1.035515,0.926938,1.406770,2.958448,2.168657,2.258597
1,CHUM-002,1.466768,1.838391,2.371688,3.293295,1.916249,1.238605,0.696015,2.323876,1.077772,...,1.701974,1.572738,2.189702,2.301615,1.561311,0.881437,1.603991,2.333146,2.828462,2.524821
2,CHUM-006,0.920737,1.962274,2.442925,3.095444,2.028646,1.232955,0.884234,1.604288,1.537089,...,1.339165,2.102762,1.705221,2.304699,1.486949,0.962173,1.284526,1.538578,2.604786,2.167278
3,CHUM-007,0.963474,1.635504,2.489746,3.565609,1.678723,1.169526,0.722139,1.875825,1.809149,...,1.771112,1.861920,1.900409,2.209949,1.010364,0.676812,1.444379,1.862758,2.346962,2.265662
4,CHUM-008,1.227698,2.024015,2.388995,3.462065,2.100064,1.511315,0.895797,1.383170,1.518909,...,1.600441,2.166911,1.210388,2.235682,1.213265,1.158197,1.247242,1.476147,1.647373,2.478250


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,1.587701,1.876628,2.530490,2.637568,2.058125,1.453559,1.032105,1.939603,1.060274,...,1.818623,1.717797,1.650555,1.965334,1.035515,0.926938,1.406770,2.958448,2.168657,2.258597
1,CHUM-002,1.466768,1.838391,2.371688,3.293295,1.916249,1.238605,0.696015,2.323876,1.077772,...,1.701974,1.572738,2.189702,2.301615,1.561311,0.881437,1.603991,2.333146,2.828462,2.524821
2,CHUM-006,0.920737,1.962274,2.442925,3.095444,2.028646,1.232955,0.884234,1.604288,1.537089,...,1.339165,2.102762,1.705221,2.304699,1.486949,0.962173,1.284526,1.538578,2.604786,2.167278
3,CHUM-007,0.963474,1.635504,2.489746,3.565609,1.678723,1.169526,0.722139,1.875825,1.809149,...,1.771112,1.861920,1.900409,2.209949,1.010364,0.676812,1.444379,1.862758,2.346962,2.265662
4,CHUM-008,1.227698,2.024015,2.388995,3.462065,2.100064,1.511315,0.895797,1.383170,1.518909,...,1.600441,2.166911,1.210388,2.235682,1.213265,1.158197,1.247242,1.476147,1.647373,2.478250


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1016,1017,1018,1019,1020,1021,1022,1023,Relapse,RFS
0,CHUM-001,1.587701,1.876628,2.530490,2.637568,2.058125,1.453559,1.032105,1.939603,1.060274,...,1.650555,1.965334,1.035515,0.926938,1.406770,2.958448,2.168657,2.258597,0,1704
1,CHUM-002,1.466768,1.838391,2.371688,3.293295,1.916249,1.238605,0.696015,2.323876,1.077772,...,2.189702,2.301615,1.561311,0.881437,1.603991,2.333146,2.828462,2.524821,1,439
2,CHUM-006,0.920737,1.962274,2.442925,3.095444,2.028646,1.232955,0.884234,1.604288,1.537089,...,1.705221,2.304699,1.486949,0.962173,1.284526,1.538578,2.604786,2.167278,0,1186
3,CHUM-007,0.963474,1.635504,2.489746,3.565609,1.678723,1.169526,0.722139,1.875825,1.809149,...,1.900409,2.209949,1.010364,0.676812,1.444379,1.862758,2.346962,2.265662,0,1702
4,CHUM-008,1.227698,2.024015,2.388995,3.462065,2.100064,1.511315,0.895797,1.383170,1.518909,...,1.210388,2.235682,1.213265,1.158197,1.247242,1.476147,1.647373,2.478250,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)